In [52]:
import pandas as pd
from altair import *
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import itables
import itertools
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import issparse
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scipy.sparse import issparse

In [10]:
df = pd.read_csv("posRating.csv")
df = df[df["avgPay"] > 1000]
df["year"] = df["year"].fillna(2024)
df.head()

,name_x,avgPay,firstName,lastName,department,numEvals,courses,overallRating,materialClear,studentDifficulties,name_y,year,dept,position,education
0,"Abercromby, Kira Jorgensen",123885.33,Kira,Abercromby,AERO,20,"['AERO 451', 'MATH 142', 'AERO 452', 'AERO 121...",3.39,3.34,3.42,"Abercromby, Kira J",2008.0,Aerospace Engineering,Professor,"B.S., University of California, Los Angeles, 1..."
1,"Abney, Keith A",99122.78,Keith,Abney,PHIL,138,"['PHIL 331', 'PHIL 335', 'PHIL 339', 'PHIL 126...",2.31,2.61,2.05,NaN,2024.0,NaN,Lecturer/TA,NaN
2,"Adams, Nikki L",116394.67,Nikki,Adams,BIO,27,"['BIO 160', 'MSC 440', 'ZOO 336', 'BIO 336', '...",2.82,2.68,2.75,"Adams, Nikki L",2002.0,Biological Sciences,Professor,"B.A., University of California, Santa Barbara,..."
3,"Adan, Elizabeth",130039.99,Elizabeth,Adan,ART,37,"['ART 118', 'ART 312', 'WGS 210', 'ART 112', '...",3.56,3.66,3.48,"Adan, Elizabeth",2007.0,"Women's, Gender and Queer Studies, Interdiscip...",Professor and WGQS Department Chair,"B.A., University of California, Davis, 1993; M..."
4,"Agbo, Samuel O",85824.17,Samuel,Agbo,EE,80,"['EE 211', 'EE 301', 'EE 418', 'EE 212', 'EE 4...",1.23,1.28,1.01,"Agbo, Samuel O",1991.0,Electrical Engineering,Professor,"B.Sc., University of Nigeria, 1975; M.S.E., Un..."


In [39]:
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.neural_network import MLPRegressor

In [12]:
deptNums = df["department"].value_counts()
def clean(d):
  if deptNums.loc[d] > 3:
    return d
  else:
    return "OTHER"

df["deptCleaned"] = df["department"].map(clean)

In [13]:
df.set_index("name_x", inplace=True)
valuable = ["avgPay", "numEvals", "overallRating", "materialClear", "studentDifficulties", "year", "position", "education", "deptCleaned"]
df = df[valuable]
df["education"] = df["education"].fillna("")
itables.show(df)

In [45]:
# ignore convergence warnings
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

In [46]:
tfidf = TfidfVectorizer(max_features=60)
one_hot = OneHotEncoder()
scaler = StandardScaler()

models = [LinearRegression(), KNeighborsRegressor(), RandomForestRegressor(), SVR(), GradientBoostingRegressor(), Ridge(), Lasso(), ElasticNet()]  # MLPRegressor()
# model = RandomForestRegressor(n_estimators=50, random_state=42)

best_features = pd.DataFrame()

for model in models:
    print(f"Model: {model}")
    all_combinations = []
    # remove the target variable from the list of features
    if "avgPay" in valuable:
        valuable.remove("avgPay")
    for r in range(1, len(valuable) + 1):
        combinations_object = itertools.combinations(valuable, r)
        combinations_list = list(combinations_object)
        all_combinations += combinations_list

    len(all_combinations)
    feature_scores = {"mse": [], "features": [], "model": []}
    for i, features in enumerate(all_combinations):
        if (i + 1) % 50 == 0:
            print(f'Combination {i + 1}/{len(all_combinations)}')
        features = list(features)
        X = df[features].copy()
        y = df["avgPay"]
        transformers = []
        X_copy = X.copy()  # create a copy of X
        if 'position' in X.columns:
            transformers.append(('position_ohe', one_hot, ['position']))
            X_copy = X_copy.drop(columns=['position'])  # drop the column from the copy
        if 'deptCleaned' in X.columns:
            transformers.append(('deptCleaned_ohe', one_hot, ['deptCleaned']))
            X_copy = X_copy.drop(columns=['deptCleaned'])  # drop the column from the copy
        if 'education' in X.columns:
            transformers.append(('education_tfidf', tfidf, 'education'))
            X_copy = X_copy.drop(columns=['education'])  # drop the column from the copy

        transformers.append(('pt', 'passthrough', list(X_copy.columns)))  # use the columns from the copy
        preprocessor = ColumnTransformer(transformers=transformers)
        X_transformed = preprocessor.fit_transform(X)
        if issparse(X_transformed):
            X_transformed = X_transformed.toarray()

        X_scaled = pd.DataFrame(scaler.fit_transform(X_transformed))

        mse = -cross_val_score(model, X_scaled, y, cv=5, scoring='neg_mean_squared_error').mean()

        feature_scores["mse"].append(mse)
        feature_scores["features"].append(features)
        feature_scores["model"].append(model)

    feature_scores_df = pd.DataFrame(feature_scores)
    best_model_features = feature_scores_df.loc[feature_scores_df.groupby("model")["mse"].idxmin()]
    best_model_features = best_model_features.sort_values(by="mse")

    best_features = pd.concat([best_features, best_model_features])



Model: LinearRegression()
Combination 50/255
Combination 100/255
Combination 150/255
Combination 200/255
Combination 250/255
Model: KNeighborsRegressor()
Combination 50/255
Combination 100/255
Combination 150/255
Combination 200/255
Combination 250/255
Model: RandomForestRegressor()
Combination 50/255
Combination 100/255
Combination 150/255
Combination 200/255
Combination 250/255
Model: SVR()
Combination 50/255
Combination 100/255
Combination 150/255
Combination 200/255
Combination 250/255
Model: GradientBoostingRegressor()
Combination 50/255
Combination 100/255
Combination 150/255
Combination 200/255
Combination 250/255
Model: Ridge()
Combination 50/255
Combination 100/255
Combination 150/255
Combination 200/255
Combination 250/255
Model: Lasso()
Combination 50/255
Combination 100/255
Combination 150/255
Combination 200/255
Combination 250/255
Model: ElasticNet()
Combination 50/255
Combination 100/255
Combination 150/255
Combination 200/255
Combination 250/255


In [47]:
# print(f'Best features: {best_features}\nMSE: {feature_scores["mse"][min_index]}')
itables.show(best_features)

216	8870943000	[materialClear, year, position, education, deptCleaned]	MLPRegressor()

GradientBoostingRegressor had best results, with numEvals, overallRating, materialClear, studentDifficulties, position, and deptCleaned as the columns. Now we can try tuning hyperparameters for optimal performance.

In [54]:
# Define the features and target variable
features = ["numEvals", "overallRating", "materialClear", "studentDifficulties", "position", "deptCleaned"]
X = df[features]
y = df["avgPay"]

# Define the transformers
one_hot = OneHotEncoder()
scaler = StandardScaler()

# Define the preprocessor
transformers = [
    ('position_ohe', one_hot, ['position']),
    ('deptCleaned_ohe', one_hot, ['deptCleaned']),
    ('scaler', scaler, ["numEvals", "overallRating", "materialClear", "studentDifficulties"])
]
preprocessor = ColumnTransformer(transformers=transformers)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameter grid for the grid search
param_grid = {
    "n_estimators": [50, 100, 200],
    "learning_rate": [0.01, 0.1, 0.2],
    "max_depth": [3, 4, 5]
}

# Initialize the GradientBoostingRegressor and GridSearchCV
gbr = GradientBoostingRegressor(random_state=42)
grid_search = GridSearchCV(gbr, param_grid, cv=5, scoring='neg_mean_squared_error')

# Create a pipeline that combines the preprocessor and the grid search
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('grid_search', grid_search)])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Print the best parameters and best score found by the grid search
print("Best parameters: ", pipeline.named_steps['grid_search'].best_params_)
print("Best score: ", -pipeline.named_steps['grid_search'].best_score_)

best_estimator = pipeline.named_steps['grid_search'].best_estimator_
X_train_preprocessed = preprocessor.transform(X_train)
cv_scores = cross_val_score(best_estimator, X_train_preprocessed, y_train, cv=5)

# print("Cross-validation scores: ", cv_scores)
print("Mean cross-validation R^2 score: ", cv_scores.mean())

Best parameters:  {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}
Best score:  855048707.2140455
Mean cross-validation R^2 score:  0.5280427422013442
